In [1]:
import pandas as pd
import numpy as np

In [2]:
labelled_df = pd.read_excel("evaluation_data_labelled.xlsx")

Compute precision, using recipes labelled as good as "true positive" and assuming all recommendations are considered a positive label:

In [3]:
labelled_df.groupby("Rec_System")["Good"].agg("mean").sort_values()

Rec_System
feature_generation    0.060000
simple_search         0.253012
sbert                 0.381526
recipe2vec            0.620000
Name: Good, dtype: float64

In [11]:
import math

# group by recommendation system and origin ID


def recommendation_set_dcg(labels):

    rankings = [
        label / math.log(index + 1.0, 2) for index, label in enumerate(labels, 1)
    ]

    total = sum(rankings)

    idcg = sum([1.0 / math.log(i + 1.0, 2) for i in range(1, len(labels) + 1)])

    return total / idcg


def aggregator(series):

    return [series]

In [12]:
ndfcg = labelled_df.groupby(["Rec_System", "Origin_Id"])["Good"].agg(aggregator)

df = ndfcg.to_frame()

# df["ndfcg"] = df["Good"].apply(recommendation_set_dcg)

In [6]:
labelled_df.columns

Index(['Column1', 'Good', 'Rec_Name', 'Rec_Description', 'Rec_Ingredients',
       'Rec_System', 'Recommendation_Id', 'Rank', 'Origin_Name',
       'Origin_Description', 'Origin_Ingredients', 'Origin_Id'],
      dtype='object')

In [13]:
pivot = labelled_df.pivot(
    index=["Rec_System", "Recommendation_Id", "Origin_Id"],
    columns="Rank",
    values="Good",
)
pivot = pivot.groupby(["Rec_System", "Origin_Id"]).agg("max")

In [14]:
pivot

Rank                            0    1    2    3    4    5    6    7    8    9
Rec_System         Origin_Id                                                  
feature_generation 2704       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
                   34431      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
                   55025      0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
                   67166      1.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
                   87667      0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
...                           ...  ...  ...  ...  ...  ...  ...  ...  ...  ...
simple_search      455125     1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
                   459476     1.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
                   461727     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
                   505623     0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
                   505798     1.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0

[100 rows x 10 columns]

In [15]:
pivot["dcfg"] = pivot.apply(lambda row: recommendation_set_dcg(row.tolist()), axis=1)

In [16]:
pivot.groupby("Rec_System").agg("mean")

Rank,0,1,2,3,4,5,6,7,8,9,dcfg
Rec_System,,,,,,,,,,,
feature_generation,0.08,0.08,0.12,0.04,0.04,0.000000,0.16,0.000000,0.00,0.08,0.011786
recipe2vec,0.80,0.56,0.64,0.68,0.52,0.600000,0.76,0.640000,0.48,0.52,0.115296
sbert,0.40,0.44,0.44,0.40,0.40,0.440000,0.36,0.291667,0.40,0.24,0.072912
simple_search,0.44,0.24,0.36,0.20,0.32,0.166667,0.16,0.240000,0.28,0.12,0.052708
